In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb

In [2]:
train = pd.read_csv('../data/train_featureV1.csv')
test = pd.read_csv('../data/test_featureV1.csv')

In [3]:
train.head()

,uid,label,voice_opp_num_unique_count,voice_opp_num_count,voice_opp_head_unique_count,voice_opp_len_5_x,voice_opp_len_6_x,voice_opp_len_7_x,voice_opp_len_8_x,voice_opp_len_9_x,...,visit_dura_x_y.10,visit_dura_y_y.10,visit_dura_x_y.11,visit_dura_y_y.11,visit_dura_x_y.12,visit_dura_y_y.12,visit_dura_x_y.13,visit_dura_y_y.13,visit_dura_x_y.14,visit_dura_y_y.14
0,u0001,0,22.0,79.0,17.0,1.0,0.0,0.0,0.0,0.0,...,47692.0,13190.0,17314.0,14337.0,34536.0,17084.0,4806.0,9402.0,3378084.0,8907.0
1,u0002,0,2.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,u0003,0,15.0,21.0,10.0,1.0,0.0,0.0,2.0,0.0,...,25104.0,17164.0,18219.0,38717.0,1039108.0,440294.0,175357.0,2184.0,452927.0,2281.0
3,u0004,0,77.0,254.0,31.0,0.0,0.0,0.0,3.0,0.0,...,145443.0,22237.0,10694.0,513625.0,110245.0,440247.0,607104.0,15360.0,1971461.0,5034.0
4,u0005,0,55.0,401.0,28.0,1.0,0.0,0.0,0.0,0.0,...,2343340.0,0.0,0.0,4844.0,82539.0,9950.0,0.0,1958.0,174052.0,160872.0


In [48]:
dtrain = lgb.Dataset(train.drop(['uid','label'],axis=1),label=train.label)
dtest = lgb.Dataset(test.drop(['uid'],axis=1))

In [45]:
def evalMetric(preds,dtrain):
    
    label = dtrain.get_label()
    
    
    pre = pd.DataFrame({'preds':preds,'label':label})
    pre= pre.sort_values(by='preds',ascending=False)
    
    auc = metrics.roc_auc_score(pre.label,pre.preds)

    pre.preds=pre.preds.map(lambda x: 1 if x>=0.38 else 0)

    f1 = metrics.f1_score(pre.label,pre.preds)
    
    
    res = 0.6*auc +0.4*f1
    
    return 'res',res,True
    

    

## lgb

In [46]:
lgb_params =  {
    'seed':0,
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'is_training_metric': False,
    'learning_rate': 0.02,
    'verbosity':-1,
    'colsample_bytree':0.7,
}

lgb_params1 = lgb_params.copy()
lgb_params1['seed'] = 1

lgb_params2 = lgb_params.copy()
lgb_params2['seed'] = 2

lgb_params3 = lgb_params.copy()
lgb_params3['seed'] = 3

lgb_params4 = lgb_params.copy()
lgb_params4['seed'] = 4

lgb_params5 = lgb_params.copy()
lgb_params5['seed'] = 5

lgb_params6 = lgb_params.copy()
lgb_params6['seed'] = 6

lgb_params7 = lgb_params.copy()
lgb_params7['seed'] = 7

lgb_params8 = lgb_params.copy()
lgb_params8['seed'] = 8

lgb_params9 = lgb_params.copy()
lgb_params9['seed'] = 9



### 本地CV

In [7]:
lgb.cv(lgb_params,dtrain,feval=evalMetric,early_stopping_rounds=100,verbose_eval=2,num_boost_round=500,nfold=3,metrics=['evalMetric'])

[2]	cv_agg's res: 0.664027 + 0.00440074
[4]	cv_agg's res: 0.682731 + 0.00675029
[6]	cv_agg's res: 0.683905 + 0.00732317
[8]	cv_agg's res: 0.683965 + 0.00707532
[10]	cv_agg's res: 0.684201 + 0.00663171
[12]	cv_agg's res: 0.686061 + 0.0056195
[14]	cv_agg's res: 0.687668 + 0.00576914
[16]	cv_agg's res: 0.802486 + 0.00662533
[18]	cv_agg's res: 0.828145 + 0.00617618
[20]	cv_agg's res: 0.840937 + 0.00903096
[22]	cv_agg's res: 0.84551 + 0.00966773
[24]	cv_agg's res: 0.8505 + 0.0114257
[26]	cv_agg's res: 0.853376 + 0.0138652
[28]	cv_agg's res: 0.855604 + 0.01409
[30]	cv_agg's res: 0.855689 + 0.0136861
[32]	cv_agg's res: 0.856509 + 0.0128601
[34]	cv_agg's res: 0.856993 + 0.0110706
[36]	cv_agg's res: 0.858892 + 0.0102104
[38]	cv_agg's res: 0.860079 + 0.0112603
[40]	cv_agg's res: 0.861815 + 0.0111514
[42]	cv_agg's res: 0.862896 + 0.0108516
[44]	cv_agg's res: 0.861952 + 0.0107404
[46]	cv_agg's res: 0.862982 + 0.00977219
[48]	cv_agg's res: 0.862684 + 0.0100811
[50]	cv_agg's res: 0.86306 + 0.0105172

[406]	cv_agg's res: 0.880703 + 0.0124813
[408]	cv_agg's res: 0.881273 + 0.0120745
[410]	cv_agg's res: 0.880657 + 0.0120903
[412]	cv_agg's res: 0.880923 + 0.0123596
[414]	cv_agg's res: 0.881026 + 0.0124203
[416]	cv_agg's res: 0.880332 + 0.0125845
[418]	cv_agg's res: 0.880512 + 0.0124075
[420]	cv_agg's res: 0.880597 + 0.012281
[422]	cv_agg's res: 0.880888 + 0.0123046
[424]	cv_agg's res: 0.881073 + 0.0126808
[426]	cv_agg's res: 0.880828 + 0.0128719
[428]	cv_agg's res: 0.880981 + 0.0128409
[430]	cv_agg's res: 0.881015 + 0.0128388
[432]	cv_agg's res: 0.880847 + 0.0126242
[434]	cv_agg's res: 0.880612 + 0.0122631
[436]	cv_agg's res: 0.881135 + 0.0125414
[438]	cv_agg's res: 0.88094 + 0.0124408
[440]	cv_agg's res: 0.880933 + 0.0133402
[442]	cv_agg's res: 0.880794 + 0.0128429
[444]	cv_agg's res: 0.881091 + 0.0125274
[446]	cv_agg's res: 0.881196 + 0.0127561
[448]	cv_agg's res: 0.881478 + 0.0128732
[450]	cv_agg's res: 0.881494 + 0.0136789
[452]	cv_agg's res: 0.881136 + 0.0136514
[454]	cv_agg's res

{'res-mean': [0.6595626934427786,
  0.6640269882992631,
  0.6686235772467861,
  0.6827311998207183,
  0.6832231622853243,
  0.6839049561412671,
  0.6839640661037337,
  0.683965060933049,
  0.6842387207517627,
  0.684200927055764,
  0.6859365511206011,
  0.6860614908292678,
  0.6862318097851485,
  0.6876681807472447,
  0.7761161150385574,
  0.8024856432414698,
  0.819316917545334,
  0.8281454528246415,
  0.8355150931620595,
  0.8409373683703293,
  0.8433778619689631,
  0.8455100421950833,
  0.849258340696767,
  0.850499860516746,
  0.8516200995955391,
  0.853376476965836,
  0.8549888591689795,
  0.855603845208461,
  0.8558393505706384,
  0.8556893725613719,
  0.8554990487665185,
  0.8565089858935883,
  0.8566360479637597,
  0.8569934827024288,
  0.8578777944128942,
  0.8588921394393575,
  0.8592799549837816,
  0.8600791883351521,
  0.8609291637261506,
  0.8618152527290835,
  0.8619614996859527,
  0.862895803157369,
  0.8617867263254436,
  0.861952188751607,
  0.8619972155113268,
  0.862

## 训练

In [8]:
model =lgb.train(lgb_params,dtrain,feval=evalMetric,verbose_eval=100,num_boost_round=392,valid_sets=[dtrain])

[100]	training's res: 0.942419
[200]	training's res: 0.979809
[300]	training's res: 0.996444


In [9]:
lgb.cv(lgb_params1,dtrain,feval=evalMetric,early_stopping_rounds=100,verbose_eval=2,num_boost_round=500,nfold=3,metrics=['evalMetric'])

[2]	cv_agg's res: 0.674992 + 0.00429868
[4]	cv_agg's res: 0.682624 + 0.00424536
[6]	cv_agg's res: 0.685059 + 0.00594536
[8]	cv_agg's res: 0.686122 + 0.00606199
[10]	cv_agg's res: 0.686087 + 0.00587678
[12]	cv_agg's res: 0.686854 + 0.00540949
[14]	cv_agg's res: 0.691096 + 0.00283053
[16]	cv_agg's res: 0.808944 + 0.00643314
[18]	cv_agg's res: 0.834293 + 0.00956639
[20]	cv_agg's res: 0.845709 + 0.0117378
[22]	cv_agg's res: 0.84969 + 0.0125035
[24]	cv_agg's res: 0.85277 + 0.0130138
[26]	cv_agg's res: 0.853179 + 0.0114848
[28]	cv_agg's res: 0.85555 + 0.0124991
[30]	cv_agg's res: 0.856922 + 0.0112973
[32]	cv_agg's res: 0.857745 + 0.0109108
[34]	cv_agg's res: 0.857816 + 0.0115348
[36]	cv_agg's res: 0.859176 + 0.0119945
[38]	cv_agg's res: 0.858557 + 0.0114731
[40]	cv_agg's res: 0.85942 + 0.0126276
[42]	cv_agg's res: 0.859692 + 0.0127294
[44]	cv_agg's res: 0.859929 + 0.0122189
[46]	cv_agg's res: 0.861078 + 0.0126834
[48]	cv_agg's res: 0.861558 + 0.0120565
[50]	cv_agg's res: 0.863704 + 0.0120874

[406]	cv_agg's res: 0.882217 + 0.0122599
[408]	cv_agg's res: 0.881917 + 0.0125411
[410]	cv_agg's res: 0.881669 + 0.0123968
[412]	cv_agg's res: 0.882017 + 0.0121734
[414]	cv_agg's res: 0.881963 + 0.0120821
[416]	cv_agg's res: 0.882146 + 0.0120311
[418]	cv_agg's res: 0.881543 + 0.0125505
[420]	cv_agg's res: 0.881566 + 0.0129558
[422]	cv_agg's res: 0.881923 + 0.0127105
[424]	cv_agg's res: 0.881643 + 0.0130199
[426]	cv_agg's res: 0.881465 + 0.0132125
[428]	cv_agg's res: 0.881187 + 0.012853
[430]	cv_agg's res: 0.881549 + 0.0133028
[432]	cv_agg's res: 0.880574 + 0.013151
[434]	cv_agg's res: 0.881165 + 0.0134086
[436]	cv_agg's res: 0.881175 + 0.0130062
[438]	cv_agg's res: 0.881084 + 0.0131963
[440]	cv_agg's res: 0.880967 + 0.0128357
[442]	cv_agg's res: 0.881333 + 0.0132586
[444]	cv_agg's res: 0.881036 + 0.0133329
[446]	cv_agg's res: 0.881169 + 0.0130623
[448]	cv_agg's res: 0.880914 + 0.0127385
[450]	cv_agg's res: 0.880931 + 0.0126185
[452]	cv_agg's res: 0.880947 + 0.0126423
[454]	cv_agg's res

{'res-mean': [0.6627354467226094,
  0.6749923847333008,
  0.6808678209640809,
  0.6826239958498266,
  0.6822650275876128,
  0.6850586305834184,
  0.6860489441626152,
  0.6861215329644752,
  0.6863127042425585,
  0.6860872016243708,
  0.6867667580515251,
  0.6868536432628956,
  0.6869310787560776,
  0.6910960174426973,
  0.7820801733705407,
  0.8089442376966448,
  0.8267928681973812,
  0.8342933206981201,
  0.8435490438754606,
  0.8457093065341156,
  0.8485179146556646,
  0.8496895347744152,
  0.8504772958175605,
  0.8527699924150038,
  0.8534600238897937,
  0.853178795054249,
  0.8547384144567536,
  0.8555495634302406,
  0.8554286200956284,
  0.8569217170438312,
  0.857207204733859,
  0.8577453494564682,
  0.8574459258418146,
  0.8578159893385712,
  0.8583452967826632,
  0.8591763341132966,
  0.8588959887643927,
  0.8585569959681895,
  0.8589609398581453,
  0.8594202797755566,
  0.8594510132473453,
  0.8596922029786079,
  0.8601536619407377,
  0.8599293560704195,
  0.8604447465685459,


In [10]:
model1 =lgb.train(lgb_params1,dtrain,feval=evalMetric,verbose_eval=100,num_boost_round=372,valid_sets=[dtrain])

[100]	training's res: 0.942872
[200]	training's res: 0.981295
[300]	training's res: 0.99602


In [11]:
lgb.cv(lgb_params2,dtrain,feval=evalMetric,early_stopping_rounds=100,verbose_eval=2,num_boost_round=500,nfold=3,metrics=['evalMetric'])

[2]	cv_agg's res: 0.671769 + 0.00790327
[4]	cv_agg's res: 0.679626 + 0.00604053
[6]	cv_agg's res: 0.681859 + 0.00771269
[8]	cv_agg's res: 0.684758 + 0.00714825
[10]	cv_agg's res: 0.686677 + 0.00635972
[12]	cv_agg's res: 0.68759 + 0.00651989
[14]	cv_agg's res: 0.6918 + 0.00487596
[16]	cv_agg's res: 0.801486 + 0.0056736
[18]	cv_agg's res: 0.828985 + 0.00960615
[20]	cv_agg's res: 0.843669 + 0.0127024
[22]	cv_agg's res: 0.848445 + 0.0121817
[24]	cv_agg's res: 0.852958 + 0.013429
[26]	cv_agg's res: 0.856531 + 0.0134862
[28]	cv_agg's res: 0.859578 + 0.0143334
[30]	cv_agg's res: 0.861156 + 0.0139026
[32]	cv_agg's res: 0.86246 + 0.013825
[34]	cv_agg's res: 0.863755 + 0.0131079
[36]	cv_agg's res: 0.862743 + 0.0120168
[38]	cv_agg's res: 0.864779 + 0.0123149
[40]	cv_agg's res: 0.86409 + 0.0107945
[42]	cv_agg's res: 0.864273 + 0.0108531
[44]	cv_agg's res: 0.863926 + 0.0104977
[46]	cv_agg's res: 0.865388 + 0.0109333
[48]	cv_agg's res: 0.865701 + 0.0118229
[50]	cv_agg's res: 0.866417 + 0.0112851
[52

[406]	cv_agg's res: 0.880417 + 0.010316
[408]	cv_agg's res: 0.880414 + 0.0103387
[410]	cv_agg's res: 0.879896 + 0.0107086
[412]	cv_agg's res: 0.880425 + 0.0109262
[414]	cv_agg's res: 0.880434 + 0.010817
[416]	cv_agg's res: 0.880179 + 0.0104654
[418]	cv_agg's res: 0.879653 + 0.0109404
[420]	cv_agg's res: 0.879398 + 0.0100938


{'res-mean': [0.6352294632875083,
  0.6717694090851086,
  0.6741876627838993,
  0.6796260208531915,
  0.6805013853234506,
  0.6818588107344582,
  0.6832964490564329,
  0.6847580182626722,
  0.6863395790932807,
  0.6866770240188657,
  0.6877297251598581,
  0.6875904121044057,
  0.6876464983069996,
  0.6918003008891119,
  0.77401673033436,
  0.801485837753618,
  0.8179532397294311,
  0.8289845213219255,
  0.8384778111697416,
  0.8436691161216548,
  0.847129115866346,
  0.8484453493205102,
  0.8511122184771054,
  0.8529578732823903,
  0.8549527607684128,
  0.8565309040518274,
  0.8590790215684079,
  0.8595780589858705,
  0.8606887690008457,
  0.8611560654277906,
  0.8614495696493845,
  0.8624597458077986,
  0.862164102279694,
  0.8637548676211884,
  0.8632479354283547,
  0.8627427759064088,
  0.8633009992241104,
  0.8647791570058786,
  0.8637355494841955,
  0.8640897466327465,
  0.8638814820189772,
  0.8642728852549273,
  0.8643069830979525,
  0.8639256265602144,
  0.8641613118325572,
  0

In [14]:
model2 =lgb.train(lgb_params3,dtrain,feval=evalMetric,verbose_eval=100,num_boost_round=322,valid_sets=[dtrain])

[100]	training's res: 0.941644
[200]	training's res: 0.980384
[300]	training's res: 0.997112


In [13]:
lgb.cv(lgb_params3,dtrain,feval=evalMetric,early_stopping_rounds=100,verbose_eval=2,num_boost_round=500,nfold=3,metrics=['evalMetric'])

[2]	cv_agg's res: 0.65872 + 0.00985763
[4]	cv_agg's res: 0.677467 + 0.00889418
[6]	cv_agg's res: 0.680154 + 0.00789225
[8]	cv_agg's res: 0.680547 + 0.00773714
[10]	cv_agg's res: 0.681758 + 0.00678834
[12]	cv_agg's res: 0.682321 + 0.00643649
[14]	cv_agg's res: 0.686445 + 0.00574124
[16]	cv_agg's res: 0.79193 + 0.00677268
[18]	cv_agg's res: 0.823191 + 0.00804002
[20]	cv_agg's res: 0.838904 + 0.010053
[22]	cv_agg's res: 0.847472 + 0.0116879
[24]	cv_agg's res: 0.850738 + 0.0118942
[26]	cv_agg's res: 0.854666 + 0.0116847
[28]	cv_agg's res: 0.858035 + 0.0112167
[30]	cv_agg's res: 0.859099 + 0.0128342
[32]	cv_agg's res: 0.859664 + 0.011899
[34]	cv_agg's res: 0.861661 + 0.0120053
[36]	cv_agg's res: 0.861674 + 0.0111191
[38]	cv_agg's res: 0.864109 + 0.0119381
[40]	cv_agg's res: 0.862653 + 0.0121053
[42]	cv_agg's res: 0.86365 + 0.0125754
[44]	cv_agg's res: 0.864052 + 0.0115111
[46]	cv_agg's res: 0.865283 + 0.0118194
[48]	cv_agg's res: 0.866258 + 0.0118905
[50]	cv_agg's res: 0.866154 + 0.0109663


[408]	cv_agg's res: 0.877188 + 0.013403
[410]	cv_agg's res: 0.877033 + 0.013484
[412]	cv_agg's res: 0.877653 + 0.0131117
[414]	cv_agg's res: 0.877538 + 0.0125945
[416]	cv_agg's res: 0.877557 + 0.0131288
[418]	cv_agg's res: 0.877179 + 0.0127803
[420]	cv_agg's res: 0.877757 + 0.0129492
[422]	cv_agg's res: 0.877648 + 0.013905
[424]	cv_agg's res: 0.87732 + 0.0141819
[426]	cv_agg's res: 0.877322 + 0.0137544


{'res-mean': [0.6448972529527793,
  0.6587204120008706,
  0.6727591016323323,
  0.6774672339201754,
  0.6793983245408611,
  0.6801543273441627,
  0.6804744194332614,
  0.680547157468444,
  0.6817737996865509,
  0.6817580305797767,
  0.6822610695128941,
  0.6823209745533744,
  0.6851178306928084,
  0.6864449912993957,
  0.7690232677055424,
  0.791929850859154,
  0.811721197478643,
  0.8231908322305519,
  0.8329248521123427,
  0.8389039760443845,
  0.8438140950974757,
  0.8474715949784736,
  0.8503962377379316,
  0.8507380778811289,
  0.854483165199246,
  0.8546664831263792,
  0.8575010015183162,
  0.858034589012249,
  0.8577902579636577,
  0.8590986069915822,
  0.8579042066116235,
  0.8596635893433748,
  0.8607930280169707,
  0.8616612288572956,
  0.8613932764658768,
  0.8616736818277078,
  0.862642472588977,
  0.8641091705862735,
  0.8626161838033969,
  0.8626526417715379,
  0.8626935727800179,
  0.8636504424049355,
  0.8637066332369118,
  0.8640521411111438,
  0.8647439555891103,
  0.

In [16]:
model3 =lgb.train(lgb_params3,dtrain,feval=evalMetric,verbose_eval=100,num_boost_round=324,valid_sets=[dtrain])

[100]	training's res: 0.941644
[200]	training's res: 0.980384
[300]	training's res: 0.997112


In [15]:
lgb.cv(lgb_params4,dtrain,feval=evalMetric,early_stopping_rounds=100,verbose_eval=2,num_boost_round=500,nfold=3,metrics=['evalMetric'])

[2]	cv_agg's res: 0.668189 + 0.00508218
[4]	cv_agg's res: 0.678967 + 0.00474724
[6]	cv_agg's res: 0.679295 + 0.00553716
[8]	cv_agg's res: 0.681138 + 0.00562472
[10]	cv_agg's res: 0.681397 + 0.00543236
[12]	cv_agg's res: 0.686102 + 0.0048378
[14]	cv_agg's res: 0.687454 + 0.00498513
[16]	cv_agg's res: 0.798631 + 0.00466109
[18]	cv_agg's res: 0.82495 + 0.00629189
[20]	cv_agg's res: 0.838263 + 0.0101716
[22]	cv_agg's res: 0.847387 + 0.011938
[24]	cv_agg's res: 0.852868 + 0.0136458
[26]	cv_agg's res: 0.858502 + 0.0120885
[28]	cv_agg's res: 0.857828 + 0.0112788
[30]	cv_agg's res: 0.859583 + 0.0114924
[32]	cv_agg's res: 0.862188 + 0.00996643
[34]	cv_agg's res: 0.863621 + 0.00872637
[36]	cv_agg's res: 0.864251 + 0.00894469
[38]	cv_agg's res: 0.864351 + 0.0102464
[40]	cv_agg's res: 0.863483 + 0.00890938
[42]	cv_agg's res: 0.864218 + 0.0101582
[44]	cv_agg's res: 0.864169 + 0.00986401
[46]	cv_agg's res: 0.864216 + 0.0109157
[48]	cv_agg's res: 0.865449 + 0.0108344
[50]	cv_agg's res: 0.866316 + 0.0

[406]	cv_agg's res: 0.879953 + 0.0133422
[408]	cv_agg's res: 0.880748 + 0.0133059
[410]	cv_agg's res: 0.880758 + 0.0133299
[412]	cv_agg's res: 0.880959 + 0.0131748
[414]	cv_agg's res: 0.880802 + 0.0133504
[416]	cv_agg's res: 0.880627 + 0.0134984
[418]	cv_agg's res: 0.88101 + 0.0135796
[420]	cv_agg's res: 0.881248 + 0.0138898
[422]	cv_agg's res: 0.881194 + 0.0138349
[424]	cv_agg's res: 0.881257 + 0.0139027
[426]	cv_agg's res: 0.881167 + 0.0144942
[428]	cv_agg's res: 0.881545 + 0.0137571
[430]	cv_agg's res: 0.881303 + 0.013371
[432]	cv_agg's res: 0.881331 + 0.0127046
[434]	cv_agg's res: 0.881291 + 0.0132586


{'res-mean': [0.6576558705523863,
  0.6681887224118942,
  0.6761887363355775,
  0.678967375120604,
  0.6790485473376435,
  0.679295206992486,
  0.6799696946634217,
  0.681138310824248,
  0.6815486385556194,
  0.6813974493124144,
  0.6815652762860163,
  0.6861015523005439,
  0.6860468597158337,
  0.6874542925251172,
  0.7708830818758958,
  0.7986305131543753,
  0.8164736795585278,
  0.8249504102270526,
  0.8311612146518123,
  0.838263361538199,
  0.8437651860875371,
  0.8473870918134345,
  0.8506948186782125,
  0.8528683823463278,
  0.8559516429681434,
  0.8585024044979295,
  0.8585926693856377,
  0.8578275083774137,
  0.8585349481516745,
  0.8595833567556813,
  0.8612454605398522,
  0.8621881143733097,
  0.8625783739194546,
  0.8636205807483748,
  0.8636014366590405,
  0.86425139334259,
  0.8640744254627709,
  0.8643507583696287,
  0.8631958102997195,
  0.8634831151043733,
  0.8639780944986738,
  0.8642176563080354,
  0.864710424196013,
  0.8641691736861966,
  0.8650601123489728,
  0.8

In [18]:
model4 =lgb.train(lgb_params4,dtrain,feval=evalMetric,verbose_eval=100,num_boost_round=334,valid_sets=[dtrain])

[100]	training's res: 0.942503
[200]	training's res: 0.98044
[300]	training's res: 0.996223


In [17]:
lgb.cv(lgb_params5,dtrain,feval=evalMetric,early_stopping_rounds=100,verbose_eval=2,num_boost_round=500,nfold=3,metrics=['evalMetric'])

[2]	cv_agg's res: 0.667139 + 0.00377811
[4]	cv_agg's res: 0.680053 + 0.0054663
[6]	cv_agg's res: 0.680798 + 0.00554735
[8]	cv_agg's res: 0.680888 + 0.00548353
[10]	cv_agg's res: 0.681566 + 0.00531831
[12]	cv_agg's res: 0.685615 + 0.00656169
[14]	cv_agg's res: 0.68691 + 0.00676554
[16]	cv_agg's res: 0.804165 + 0.00608146
[18]	cv_agg's res: 0.830095 + 0.0065716
[20]	cv_agg's res: 0.840558 + 0.00881806
[22]	cv_agg's res: 0.850672 + 0.00990767
[24]	cv_agg's res: 0.853197 + 0.0115701
[26]	cv_agg's res: 0.856597 + 0.0121174
[28]	cv_agg's res: 0.85738 + 0.0138393
[30]	cv_agg's res: 0.857838 + 0.014105
[32]	cv_agg's res: 0.85908 + 0.0134644
[34]	cv_agg's res: 0.859613 + 0.0137877
[36]	cv_agg's res: 0.861642 + 0.0118039
[38]	cv_agg's res: 0.86183 + 0.0123051
[40]	cv_agg's res: 0.863321 + 0.0120833
[42]	cv_agg's res: 0.864129 + 0.0121875
[44]	cv_agg's res: 0.86559 + 0.0114537
[46]	cv_agg's res: 0.865855 + 0.0116975
[48]	cv_agg's res: 0.866293 + 0.011437
[50]	cv_agg's res: 0.865896 + 0.00993523
[

[408]	cv_agg's res: 0.881396 + 0.014385
[410]	cv_agg's res: 0.881675 + 0.0138474
[412]	cv_agg's res: 0.881499 + 0.013721
[414]	cv_agg's res: 0.881846 + 0.0137911
[416]	cv_agg's res: 0.882127 + 0.0139479
[418]	cv_agg's res: 0.881481 + 0.0138338
[420]	cv_agg's res: 0.8816 + 0.0143483
[422]	cv_agg's res: 0.88127 + 0.0143545
[424]	cv_agg's res: 0.880373 + 0.0146345
[426]	cv_agg's res: 0.881567 + 0.014133
[428]	cv_agg's res: 0.881179 + 0.0149307
[430]	cv_agg's res: 0.88105 + 0.0141168
[432]	cv_agg's res: 0.881007 + 0.0141429
[434]	cv_agg's res: 0.881162 + 0.0145574
[436]	cv_agg's res: 0.881347 + 0.0142722
[438]	cv_agg's res: 0.880802 + 0.0137474
[440]	cv_agg's res: 0.880296 + 0.013532
[442]	cv_agg's res: 0.880287 + 0.0140944
[444]	cv_agg's res: 0.880888 + 0.0141889
[446]	cv_agg's res: 0.880798 + 0.0136698
[448]	cv_agg's res: 0.880659 + 0.0138165
[450]	cv_agg's res: 0.880469 + 0.0128488
[452]	cv_agg's res: 0.880022 + 0.0131582


{'res-mean': [0.6650589065579601,
  0.6671389098996441,
  0.6790711431548976,
  0.6800526985588399,
  0.6796923807704062,
  0.6807979726286931,
  0.6809216875887626,
  0.6808877732451196,
  0.6813752915556556,
  0.6815660435166636,
  0.6814745495661468,
  0.6856151634149278,
  0.6859441107880465,
  0.6869097724428735,
  0.7746531456530734,
  0.8041651490882158,
  0.8190507296883517,
  0.8300953773677877,
  0.8386918365171233,
  0.8405583560298063,
  0.8465837207879104,
  0.8506719521048066,
  0.8521575579395803,
  0.853196547921819,
  0.8551640458463369,
  0.8565966787611718,
  0.8565263791050374,
  0.8573796845249267,
  0.8567981112009058,
  0.857838167280053,
  0.858634734080054,
  0.8590798038219282,
  0.8596385836591769,
  0.8596126341568525,
  0.859528894359769,
  0.8616417651249928,
  0.861297396392032,
  0.8618298088247799,
  0.8627940804703497,
  0.8633209592975799,
  0.8641155107646489,
  0.8641287326019057,
  0.8643664418398741,
  0.865590334822225,
  0.8657824680439555,
  0.

In [20]:
model5 =lgb.train(lgb_params5,dtrain,feval=evalMetric,verbose_eval=100,num_boost_round=352,valid_sets=[dtrain])

[100]	training's res: 0.942197
[200]	training's res: 0.98001
[300]	training's res: 0.997553


In [19]:
lgb.cv(lgb_params6,dtrain,feval=evalMetric,early_stopping_rounds=100,verbose_eval=2,num_boost_round=500,nfold=3,metrics=['evalMetric'])

[2]	cv_agg's res: 0.670325 + 0.00629756
[4]	cv_agg's res: 0.681323 + 0.00491621
[6]	cv_agg's res: 0.685911 + 0.005187
[8]	cv_agg's res: 0.686868 + 0.00588455
[10]	cv_agg's res: 0.687355 + 0.00568661
[12]	cv_agg's res: 0.687923 + 0.00479679
[14]	cv_agg's res: 0.693405 + 0.00346694
[16]	cv_agg's res: 0.802572 + 0.00535544
[18]	cv_agg's res: 0.828028 + 0.00751399
[20]	cv_agg's res: 0.84101 + 0.00954827
[22]	cv_agg's res: 0.85066 + 0.0118966
[24]	cv_agg's res: 0.854334 + 0.0120995
[26]	cv_agg's res: 0.85601 + 0.0112732
[28]	cv_agg's res: 0.857972 + 0.0113468
[30]	cv_agg's res: 0.859539 + 0.0111073
[32]	cv_agg's res: 0.859413 + 0.00934281
[34]	cv_agg's res: 0.861468 + 0.0110823
[36]	cv_agg's res: 0.861216 + 0.0106376
[38]	cv_agg's res: 0.862222 + 0.00952601
[40]	cv_agg's res: 0.862225 + 0.0103882
[42]	cv_agg's res: 0.862136 + 0.0103961
[44]	cv_agg's res: 0.863313 + 0.0113439
[46]	cv_agg's res: 0.862349 + 0.0110204
[48]	cv_agg's res: 0.862969 + 0.0119006
[50]	cv_agg's res: 0.864062 + 0.01197

{'res-mean': [0.6638182572930412,
  0.6703254056763054,
  0.6733316756179651,
  0.6813227284019341,
  0.685063398373869,
  0.6859106295066534,
  0.6855251171704486,
  0.6868682638443127,
  0.6872305509413784,
  0.6873550111755176,
  0.6881002618607234,
  0.6879229419698136,
  0.6879392660603205,
  0.693405174768861,
  0.7804813787267281,
  0.802572152184558,
  0.8199928007654668,
  0.8280282414204354,
  0.8355676818608658,
  0.8410098270401885,
  0.8462167184958855,
  0.8506597702843668,
  0.8522277963969378,
  0.8543341598500955,
  0.8544905540127591,
  0.8560096029444613,
  0.8567207940602307,
  0.8579718964231485,
  0.8578385832913611,
  0.8595393638467929,
  0.8589792451461596,
  0.8594130564592538,
  0.8598393901295416,
  0.8614678236460156,
  0.8612702882969715,
  0.8612158670079162,
  0.8613604932049533,
  0.8622220618608422,
  0.8626731824948842,
  0.8622249587442554,
  0.8615822397971106,
  0.8621358198186687,
  0.8616863020680575,
  0.8633125850137815,
  0.8631819909357864,
 

In [22]:
model6 =lgb.train(lgb_params6,dtrain,feval=evalMetric,verbose_eval=100,num_boost_round=212,valid_sets=[dtrain])

[100]	training's res: 0.942245
[200]	training's res: 0.982168


In [21]:
lgb.cv(lgb_params7,dtrain,feval=evalMetric,early_stopping_rounds=100,verbose_eval=2,num_boost_round=500,nfold=3,metrics=['evalMetric'])

[2]	cv_agg's res: 0.664724 + 0.00655441
[4]	cv_agg's res: 0.677841 + 0.00748257
[6]	cv_agg's res: 0.681267 + 0.00678715
[8]	cv_agg's res: 0.685061 + 0.00595259
[10]	cv_agg's res: 0.68601 + 0.00514727
[12]	cv_agg's res: 0.686261 + 0.00530033
[14]	cv_agg's res: 0.691101 + 0.00260526
[16]	cv_agg's res: 0.801596 + 0.00623113
[18]	cv_agg's res: 0.834735 + 0.0108354
[20]	cv_agg's res: 0.846012 + 0.0115274
[22]	cv_agg's res: 0.852207 + 0.0118447
[24]	cv_agg's res: 0.855829 + 0.0121134
[26]	cv_agg's res: 0.859358 + 0.0103086
[28]	cv_agg's res: 0.861024 + 0.0109873
[30]	cv_agg's res: 0.863109 + 0.0111787
[32]	cv_agg's res: 0.863086 + 0.00976939
[34]	cv_agg's res: 0.863501 + 0.00914152
[36]	cv_agg's res: 0.863315 + 0.00905543
[38]	cv_agg's res: 0.864019 + 0.00850421
[40]	cv_agg's res: 0.862102 + 0.00850304
[42]	cv_agg's res: 0.865308 + 0.00866413
[44]	cv_agg's res: 0.863904 + 0.0101236
[46]	cv_agg's res: 0.864451 + 0.0109719
[48]	cv_agg's res: 0.864607 + 0.0114053
[50]	cv_agg's res: 0.86566 + 0.

[406]	cv_agg's res: 0.881076 + 0.0125066
[408]	cv_agg's res: 0.88083 + 0.0121295
[410]	cv_agg's res: 0.880802 + 0.0126794
[412]	cv_agg's res: 0.881488 + 0.0127318
[414]	cv_agg's res: 0.881445 + 0.0122428
[416]	cv_agg's res: 0.881346 + 0.012117
[418]	cv_agg's res: 0.881186 + 0.0118794
[420]	cv_agg's res: 0.881023 + 0.0120093
[422]	cv_agg's res: 0.881112 + 0.0117965


{'res-mean': [0.6603729075711757,
  0.6647235148401238,
  0.6742728134256563,
  0.6778405348291777,
  0.6758732501294848,
  0.6812674249602496,
  0.6822521833468588,
  0.685061322316999,
  0.6857521819187887,
  0.6860101338592326,
  0.6862142752267454,
  0.686261357269014,
  0.6861491411291802,
  0.6911013186410818,
  0.7725260191459732,
  0.8015964935794906,
  0.821472188157912,
  0.8347345102638307,
  0.8431180542884794,
  0.8460119573804668,
  0.850476300215537,
  0.8522074493855264,
  0.8540324209860937,
  0.8558289148681855,
  0.8581098174032862,
  0.8593580177790378,
  0.8606314566521102,
  0.8610244544556362,
  0.8617826969780807,
  0.8631090773038531,
  0.8624568423018291,
  0.8630864060505042,
  0.8629934099753681,
  0.8635012494993554,
  0.8637365309268116,
  0.863314695979576,
  0.8634040966792913,
  0.8640187306997689,
  0.8635016169268818,
  0.8621022956607401,
  0.8639411482871572,
  0.8653084690914049,
  0.8645752820174316,
  0.8639038723733655,
  0.8644169547652295,
  0

In [24]:
model7 =lgb.train(lgb_params7,dtrain,feval=evalMetric,verbose_eval=100,num_boost_round=322,valid_sets=[dtrain])

[100]	training's res: 0.941901
[200]	training's res: 0.980647
[300]	training's res: 0.996673


In [23]:
lgb.cv(lgb_params8,dtrain,feval=evalMetric,early_stopping_rounds=100,verbose_eval=2,num_boost_round=500,nfold=3,metrics=['evalMetric'])

[2]	cv_agg's res: 0.665207 + 0.00509667
[4]	cv_agg's res: 0.677069 + 0.00546404
[6]	cv_agg's res: 0.680322 + 0.004377
[8]	cv_agg's res: 0.681314 + 0.00434107
[10]	cv_agg's res: 0.685203 + 0.00618905
[12]	cv_agg's res: 0.684908 + 0.00627721
[14]	cv_agg's res: 0.686278 + 0.00658564
[16]	cv_agg's res: 0.79916 + 0.00689348
[18]	cv_agg's res: 0.831645 + 0.010206
[20]	cv_agg's res: 0.84358 + 0.0116708
[22]	cv_agg's res: 0.84871 + 0.0119628
[24]	cv_agg's res: 0.8543 + 0.0115603
[26]	cv_agg's res: 0.857479 + 0.0119382
[28]	cv_agg's res: 0.859117 + 0.0116456
[30]	cv_agg's res: 0.858442 + 0.00955476
[32]	cv_agg's res: 0.860641 + 0.00951673
[34]	cv_agg's res: 0.861263 + 0.010042
[36]	cv_agg's res: 0.862215 + 0.00949588
[38]	cv_agg's res: 0.861821 + 0.010163
[40]	cv_agg's res: 0.861584 + 0.00955044
[42]	cv_agg's res: 0.863325 + 0.0101687
[44]	cv_agg's res: 0.864387 + 0.0102779
[46]	cv_agg's res: 0.863948 + 0.0114936
[48]	cv_agg's res: 0.864342 + 0.012018
[50]	cv_agg's res: 0.86475 + 0.0117774
[52]

[408]	cv_agg's res: 0.881679 + 0.013216
[410]	cv_agg's res: 0.881571 + 0.0129488
[412]	cv_agg's res: 0.881935 + 0.0119627
[414]	cv_agg's res: 0.881615 + 0.0118911
[416]	cv_agg's res: 0.881199 + 0.0121771
[418]	cv_agg's res: 0.880703 + 0.0112739
[420]	cv_agg's res: 0.880706 + 0.0112527
[422]	cv_agg's res: 0.880279 + 0.0116657
[424]	cv_agg's res: 0.881036 + 0.0111122
[426]	cv_agg's res: 0.881066 + 0.011114
[428]	cv_agg's res: 0.880969 + 0.011085
[430]	cv_agg's res: 0.880849 + 0.0114738
[432]	cv_agg's res: 0.880672 + 0.011347
[434]	cv_agg's res: 0.880903 + 0.0123503
[436]	cv_agg's res: 0.880645 + 0.0126346
[438]	cv_agg's res: 0.880699 + 0.0126053


{'res-mean': [0.6487833002797356,
  0.6652072355553157,
  0.6740415680021744,
  0.6770685958349033,
  0.6791469195876715,
  0.6803219224348863,
  0.6808550072483249,
  0.6813136365938083,
  0.6817691697048623,
  0.6852032462415639,
  0.6849226349347477,
  0.6849082464147175,
  0.6855536605425433,
  0.6862775443171056,
  0.7709966292353322,
  0.799160399044282,
  0.8188147031513283,
  0.8316450616053791,
  0.8373999913919361,
  0.8435800383270227,
  0.8455307596565103,
  0.8487098104644826,
  0.8519307117390764,
  0.8542998620110991,
  0.8558710980511611,
  0.8574786863632604,
  0.8590329348139155,
  0.859116814615397,
  0.8576426531628183,
  0.8584423058213377,
  0.8599565590392432,
  0.8606413830210039,
  0.8603438170078547,
  0.8612634059770036,
  0.8619023450492884,
  0.8622154534701824,
  0.8625079272871822,
  0.8618208958159318,
  0.8614335422132768,
  0.8615840992075213,
  0.8620042501143123,
  0.8633254403687497,
  0.862695975041052,
  0.8643866557275016,
  0.8648698015698034,
 

In [26]:
model8 =lgb.train(lgb_params8,dtrain,feval=evalMetric,verbose_eval=100,num_boost_round=174,valid_sets=[dtrain])

[100]	training's res: 0.941541


In [25]:
lgb.cv(lgb_params9,dtrain,feval=evalMetric,early_stopping_rounds=100,verbose_eval=2,num_boost_round=500,nfold=3,metrics=['evalMetric'])

[2]	cv_agg's res: 0.676788 + 0.0050317
[4]	cv_agg's res: 0.681826 + 0.00349161
[6]	cv_agg's res: 0.684049 + 0.00353132
[8]	cv_agg's res: 0.684477 + 0.00379498
[10]	cv_agg's res: 0.685265 + 0.00355069
[12]	cv_agg's res: 0.684778 + 0.00360545
[14]	cv_agg's res: 0.684868 + 0.00372204
[16]	cv_agg's res: 0.809448 + 0.00199831
[18]	cv_agg's res: 0.831634 + 0.0067652
[20]	cv_agg's res: 0.841736 + 0.0113128
[22]	cv_agg's res: 0.847038 + 0.0122462
[24]	cv_agg's res: 0.849078 + 0.010337
[26]	cv_agg's res: 0.851015 + 0.0106319
[28]	cv_agg's res: 0.85176 + 0.0112819
[30]	cv_agg's res: 0.853326 + 0.0109001
[32]	cv_agg's res: 0.854674 + 0.0104082
[34]	cv_agg's res: 0.853779 + 0.0109265
[36]	cv_agg's res: 0.855031 + 0.0102378
[38]	cv_agg's res: 0.857426 + 0.00977002
[40]	cv_agg's res: 0.858315 + 0.0104744
[42]	cv_agg's res: 0.859083 + 0.0112045
[44]	cv_agg's res: 0.859124 + 0.0112302
[46]	cv_agg's res: 0.859685 + 0.0114956
[48]	cv_agg's res: 0.859569 + 0.0114444
[50]	cv_agg's res: 0.860184 + 0.010958

[406]	cv_agg's res: 0.881226 + 0.0109851
[408]	cv_agg's res: 0.88148 + 0.0113645
[410]	cv_agg's res: 0.881768 + 0.01112
[412]	cv_agg's res: 0.881862 + 0.0112516
[414]	cv_agg's res: 0.881808 + 0.0105352
[416]	cv_agg's res: 0.881808 + 0.0105286
[418]	cv_agg's res: 0.881877 + 0.0112328
[420]	cv_agg's res: 0.882875 + 0.0116117
[422]	cv_agg's res: 0.882136 + 0.0121422
[424]	cv_agg's res: 0.882582 + 0.012168
[426]	cv_agg's res: 0.882148 + 0.0115701
[428]	cv_agg's res: 0.882506 + 0.0116888
[430]	cv_agg's res: 0.881648 + 0.0113766
[432]	cv_agg's res: 0.881252 + 0.0107423
[434]	cv_agg's res: 0.88169 + 0.0109723
[436]	cv_agg's res: 0.881572 + 0.0114237
[438]	cv_agg's res: 0.882017 + 0.0111044
[440]	cv_agg's res: 0.881839 + 0.0112102
[442]	cv_agg's res: 0.881268 + 0.0116392
[444]	cv_agg's res: 0.88125 + 0.0116484
[446]	cv_agg's res: 0.881778 + 0.0117974
[448]	cv_agg's res: 0.881684 + 0.011679
[450]	cv_agg's res: 0.881437 + 0.0113038
[452]	cv_agg's res: 0.881469 + 0.0112984
[454]	cv_agg's res: 0.8

{'res-mean': [0.653440086486578,
  0.676787930439933,
  0.6785091186799187,
  0.6818255340241032,
  0.6833078841490415,
  0.6840494843301208,
  0.6844199855077431,
  0.6844772880690225,
  0.6846053175858029,
  0.6852648945985873,
  0.6853827316222322,
  0.6847779293081876,
  0.6844546736868574,
  0.6848684369824752,
  0.7782830229404242,
  0.8094481414125719,
  0.8254812643122156,
  0.8316342387376837,
  0.8375381779156322,
  0.8417363167080176,
  0.8453438900984599,
  0.8470380796622224,
  0.8471388723043214,
  0.8490777899539715,
  0.8498374974603881,
  0.8510149926819549,
  0.8513356739240051,
  0.8517597489725364,
  0.851931269595357,
  0.8533262836240704,
  0.8539623241218992,
  0.8546743251267758,
  0.8545413496813828,
  0.8537788250386334,
  0.8546489096212099,
  0.8550305469582051,
  0.8572189722541649,
  0.857425875755589,
  0.8571525326742816,
  0.8583148537048868,
  0.8596338395747519,
  0.8590831477223427,
  0.8588351497173107,
  0.8591239626733115,
  0.8592635567352306,
  

In [49]:
model9 =lgb.train(lgb_params9,dtrain,feval=evalMetric,verbose_eval=100,num_boost_round=420,valid_sets=[dtrain])

[100]	training's binary_logloss: 0.190023	training's res: 0.939374
[200]	training's binary_logloss: 0.0953637	training's res: 0.979967
[300]	training's binary_logloss: 0.0555756	training's res: 0.996458
[400]	training's binary_logloss: 0.0341713	training's res: 0.999778


## xgb

In [55]:
xgb_params = {
    'booster':'gbtree',
    'objective':'binary:logistic',
    'stratified':True,
    #'max_depth':10,
    # 'gamma':1,
    #'subsample':0.8,
    #'colsample_bytree':0.8,
    # 'lambda':1,
    'eta':0.01,
    'seed':20,
    'silent':1,
}

xgb_params2 = {
    'booster':'gbtree',
    'objective':'binary:logistic',
    'stratified':True,
    #'max_depth':10,
    # 'gamma':1,
    #'subsample':0.8,
    #'colsample_bytree':0.8,
    # 'lambda':1,
    'eta':0.01,
    'seed':6,
    'silent':1,
}


def evalMetric(preds,dtrain):
    label = dtrain.get_label()
    pre = pd.DataFrame({'preds':preds,'label':label})
    pre= pre.sort_values(by='preds',ascending=False)
    auc = metrics.roc_auc_score(pre.label,pre.preds)
    pre.preds=pre.preds.map(lambda x: 1 if x>=0.38 else 0)
    f1 = metrics.f1_score(pre.label,pre.preds)
    res = 0.6*auc +0.4*f1
    return 'res',res

In [28]:
dtrain = xgb.DMatrix(train.drop(['uid','label'],axis=1),label=train.label)
xgb.cv(xgb_params,dtrain,num_boost_round=700,nfold=3,verbose_eval=10,early_stopping_rounds=100,maximize=True,feval=evalMetric)

[0]	train-error:0.077816+0.00368735	train-res:0.689607+0.00625094	test-error:0.122627+0.0105613	test-res:0.659911+0.0148438
[10]	train-error:0.071614+0.00251175	train-res:0.695868+0.00550032	test-error:0.116427+0.0156055	test-res:0.662292+0.0156407
[20]	train-error:0.0686137+0.000746607	train-res:0.699168+0.00599593	test-error:0.109424+0.00891979	test-res:0.665534+0.014157
[30]	train-error:0.065213+0.000372941	train-res:0.827957+0.00383041	test-error:0.107824+0.00960544	test-res:0.779384+0.022404
[40]	train-error:0.0630127+0.000889994	train-res:0.876706+0.00647505	test-error:0.108823+0.00912748	test-res:0.81665+0.0253603
[50]	train-error:0.0628123+0.000561261	train-res:0.893665+0.00440405	test-error:0.109223+0.00765447	test-res:0.831192+0.0270277
[60]	train-error:0.061112+0.00110511	train-res:0.900167+0.00441875	test-error:0.108423+0.00738645	test-res:0.834127+0.0265932
[70]	train-error:0.0584117+0.00123199	train-res:0.906985+0.00481828	test-error:0.108023+0.00808192	test-res:0.838659+

[650]	train-error:0.00810167+0.00259369	train-res:0.989253+0.00110178	test-error:0.0924197+0.00723247	test-res:0.877248+0.0184445
[660]	train-error:0.00740167+0.00214157	train-res:0.98983+0.00114062	test-error:0.0924193+0.00680358	test-res:0.877069+0.0178725
[670]	train-error:0.00740167+0.00214157	train-res:0.99017+0.00114404	test-error:0.0928193+0.00736926	test-res:0.877349+0.018141
[680]	train-error:0.00680133+0.00190375	train-res:0.990611+0.000856487	test-error:0.0938197+0.00793542	test-res:0.876496+0.0185051
[690]	train-error:0.00640133+0.00183971	train-res:0.991283+0.000811524	test-error:0.0938197+0.0083639	test-res:0.876051+0.0185227
[699]	train-error:0.00600133+0.00147092	train-res:0.991521+0.00111959	test-error:0.09362+0.00851619	test-res:0.876768+0.0180734


,test-error-mean,test-error-std,test-res-mean,test-res-std,train-error-mean,train-error-std,train-res-mean,train-res-std
0,0.122627,0.010561,0.659911,0.014844,0.077816,0.003687,0.689607,0.006251
1,0.118827,0.013851,0.660474,0.015696,0.075016,0.003617,0.691667,0.005612
2,0.116026,0.012779,0.660485,0.015710,0.075416,0.004177,0.691980,0.005893
3,0.117827,0.014515,0.660924,0.015735,0.075716,0.003823,0.692885,0.005804
4,0.116227,0.014281,0.660937,0.015855,0.075015,0.003222,0.693020,0.005681
5,0.115627,0.015967,0.660832,0.015652,0.073915,0.002695,0.693503,0.005834
6,0.116827,0.015219,0.661347,0.015809,0.073915,0.003726,0.694082,0.005738
7,0.117426,0.014363,0.661318,0.015555,0.071014,0.001720,0.694333,0.005882
8,0.115426,0.015209,0.661548,0.015696,0.071214,0.001873,0.694840,0.005327
9,0.115627,0.016026,0.661801,0.015599,0.071714,0.001948,0.695123,0.005695


In [57]:
dtrain = xgb.DMatrix(train.drop(['uid','label'],axis=1),label=train.label)
xgbmodel=xgb.train(xgb_params,dtrain=dtrain,num_boost_round=620,verbose_eval=100,
                evals=[(dtrain,'train')],maximize=True,feval=evalMetric,early_stopping_rounds=100)

[0]	train-error:0.083617	train-res:0.685305
Multiple eval metrics have been passed: 'train-res' will be used for early stopping.

Will train until train-res hasn't improved in 100 rounds.
[100]	train-error:0.064213	train-res:0.90689
[200]	train-error:0.05241	train-res:0.926287
[300]	train-error:0.043009	train-res:0.942322
[400]	train-error:0.035607	train-res:0.953071
[500]	train-error:0.027405	train-res:0.960064
[600]	train-error:0.018004	train-res:0.970628
[619]	train-error:0.016603	train-res:0.973265


In [30]:
xgb.cv(xgb_params2,dtrain,num_boost_round=700,nfold=3,verbose_eval=10,early_stopping_rounds=100,maximize=True,feval=evalMetric)

[0]	train-error:0.077816+0.00368735	train-res:0.689607+0.00625094	test-error:0.122627+0.0105613	test-res:0.659911+0.0148438
[10]	train-error:0.071614+0.00251175	train-res:0.695868+0.00550032	test-error:0.116427+0.0156055	test-res:0.662292+0.0156407
[20]	train-error:0.0686137+0.000746607	train-res:0.699168+0.00599593	test-error:0.109424+0.00891979	test-res:0.665534+0.014157
[30]	train-error:0.065213+0.000372941	train-res:0.827957+0.00383041	test-error:0.107824+0.00960544	test-res:0.779384+0.022404
[40]	train-error:0.0630127+0.000889994	train-res:0.876706+0.00647505	test-error:0.108823+0.00912748	test-res:0.81665+0.0253603
[50]	train-error:0.0628123+0.000561261	train-res:0.893665+0.00440405	test-error:0.109223+0.00765447	test-res:0.831192+0.0270277
[60]	train-error:0.061112+0.00110511	train-res:0.900167+0.00441875	test-error:0.108423+0.00738645	test-res:0.834127+0.0265932
[70]	train-error:0.0584117+0.00123199	train-res:0.906985+0.00481828	test-error:0.108023+0.00808192	test-res:0.838659+

[650]	train-error:0.00810167+0.00259369	train-res:0.989253+0.00110178	test-error:0.0924197+0.00723247	test-res:0.877248+0.0184445
[660]	train-error:0.00740167+0.00214157	train-res:0.98983+0.00114062	test-error:0.0924193+0.00680358	test-res:0.877069+0.0178725
[670]	train-error:0.00740167+0.00214157	train-res:0.99017+0.00114404	test-error:0.0928193+0.00736926	test-res:0.877349+0.018141
[680]	train-error:0.00680133+0.00190375	train-res:0.990611+0.000856487	test-error:0.0938197+0.00793542	test-res:0.876496+0.0185051
[690]	train-error:0.00640133+0.00183971	train-res:0.991283+0.000811524	test-error:0.0938197+0.0083639	test-res:0.876051+0.0185227
[699]	train-error:0.00600133+0.00147092	train-res:0.991521+0.00111959	test-error:0.09362+0.00851619	test-res:0.876768+0.0180734


,test-error-mean,test-error-std,test-res-mean,test-res-std,train-error-mean,train-error-std,train-res-mean,train-res-std
0,0.122627,0.010561,0.659911,0.014844,0.077816,0.003687,0.689607,0.006251
1,0.118827,0.013851,0.660474,0.015696,0.075016,0.003617,0.691667,0.005612
2,0.116026,0.012779,0.660485,0.015710,0.075416,0.004177,0.691980,0.005893
3,0.117827,0.014515,0.660924,0.015735,0.075716,0.003823,0.692885,0.005804
4,0.116227,0.014281,0.660937,0.015855,0.075015,0.003222,0.693020,0.005681
5,0.115627,0.015967,0.660832,0.015652,0.073915,0.002695,0.693503,0.005834
6,0.116827,0.015219,0.661347,0.015809,0.073915,0.003726,0.694082,0.005738
7,0.117426,0.014363,0.661318,0.015555,0.071014,0.001720,0.694333,0.005882
8,0.115426,0.015209,0.661548,0.015696,0.071214,0.001873,0.694840,0.005327
9,0.115627,0.016026,0.661801,0.015599,0.071714,0.001948,0.695123,0.005695


In [34]:
dtrain = xgb.DMatrix(train.drop(['uid','label'],axis=1),label=train.label)
xgbmode2=xgb.train(xgb_params2,dtrain=dtrain,num_boost_round=670,verbose_eval=100,
                evals=[(dtrain,'train')],maximize=True,feval=evalMetric,early_stopping_rounds=100)

[0]	train-error:0.083617	train-res:0.685305
Multiple eval metrics have been passed: 'train-res' will be used for early stopping.

Will train until train-res hasn't improved in 100 rounds.
[100]	train-error:0.064213	train-res:0.90689
[200]	train-error:0.05241	train-res:0.926287
[300]	train-error:0.043009	train-res:0.942322
[400]	train-error:0.035607	train-res:0.953071
[500]	train-error:0.027405	train-res:0.960064
[600]	train-error:0.018004	train-res:0.970628
[669]	train-error:0.014203	train-res:0.976904


### 预测

In [35]:
pred0=model.predict(test.drop(['uid'],axis=1))

In [36]:
pred1=model1.predict(test.drop(['uid'],axis=1))

In [37]:
pred2=model2.predict(test.drop(['uid'],axis=1))

In [38]:
pred3=model3.predict(test.drop(['uid'],axis=1))

In [39]:
pred4=model4.predict(test.drop(['uid'],axis=1))

In [40]:
pred5=model5.predict(test.drop(['uid'],axis=1))

In [41]:
pred6=model6.predict(test.drop(['uid'],axis=1))

In [42]:
pred7=model7.predict(test.drop(['uid'],axis=1))

In [43]:
pred8=model8.predict(test.drop(['uid'],axis=1))

In [50]:
pred9=model9.predict(test.drop(['uid'],axis=1))

In [58]:
dtest = xgb.DMatrix(test.drop(['uid'],axis=1))
xgbpreds =xgbmodel.predict(dtest)
xgbpreds2 =xgbmode2.predict(dtest)

In [59]:
pred =pd.DataFrame({'p0':pred0,'p1':pred1,'p2':pred2,'p3':pred3,'p4':pred4,'p5':pred5,'p6':pred6,'p7':pred7,'p8':pred8,'p9':pred9,'pxgb1':xgbpreds,'xgbp2':xgbpreds2})
pred.head()

,p0,p1,p2,p3,p4,p5,p6,p7,p8,p9,pxgb1,xgbp2
0,0.869607,0.834481,0.754461,0.754012,0.850311,0.871350,0.741426,0.773067,0.695321,0.850528,0.760831,0.764900
1,0.764260,0.700824,0.702187,0.702801,0.717923,0.653535,0.704231,0.685129,0.694180,0.719616,0.697770,0.703492
2,0.004864,0.006335,0.008919,0.008945,0.006731,0.008308,0.021608,0.008920,0.030484,0.004263,0.011799,0.009348
3,0.012181,0.014158,0.020392,0.020450,0.021793,0.016129,0.044679,0.022516,0.073573,0.015311,0.024472,0.021561
4,0.038961,0.041522,0.054416,0.054567,0.038278,0.042559,0.079036,0.055714,0.100280,0.030066,0.094219,0.089362


In [60]:
# 求平均值
pred_t = pred.T
pred_mean = pred_t.mean()
res =pd.DataFrame({'uid':test.uid,'label':pred_mean})

In [61]:
res=res.sort_values(by='label',ascending=False)
res.label=res.label.map(lambda x: 1 if x>=0.38 else 0)
#res.label = res.label.map(lambda x: int(x))

In [62]:
res.to_csv('../result/lgb-baseline.csv',index=False,header=False,sep=',',columns=['uid','label'])